In [2]:
### all imports here - we will make requirements document for these
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import Image, display, clear_output
import pandas as pd
from geopy import distance
import folium

In [2]:
### this is to enable the widgets/voila in the notebook - only need once per session
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling: voila
- Writing config: /Users/a0271775/miniconda3/envs/dva_env/etc/jupyter
    - Validating...
      voila 0.2.16 OK


In [ ]:
### Add all sub-module definitions here
def distance_calc(row, user_input):
    #print('made it')
    #print(row)
    row_loc = (row['Lat Split'], row['Long Split'])
    calc = distance.distance(row_loc, user_input).miles
    return calc
def label(row):
    return_str = ""
    if not 'UNNAMED' in row['Name']:
        return_str = return_str + "Name: " + row['Name']
    return_str = return_str + " Event: " + row['Status']
    if row['Maximum Wind'] != -99:
        return_str = return_str + " Max Wind: " + str(row['Maximum Wind'])
    #print(return_str)
    return return_str

In [ ]:
### read all the csvs
storm_df = pd.read_csv('DataAddressUSA_JOIN.csv')
zip_df = pd.read_csv('zipcodes.csv')
#fixing the zip codes that start with 00
zip_df['zip'] = zip_df['zip'].apply('{:0>5}'.format)
#zip_df.head()

In [ ]:
### first widget input - zip-code from user
#zip_input = input("What is your zip code (5 digit code and press enter)?") ## actual jupyter version
zip_text = widgets.Text(placeholder='Your 5-digit zip code',layout=widgets.Layout(width='40%',margin='0% 0% 0% 10% '))
#print(zip_text)

In [ ]:
### call the html page setup here
text_0 = widgets.HTML(value="<br /><h1 style=\"color:#34568B;font-size: 35px;margin-left:20px;\">Simplified Dashboard for Hurricane Impact!</h1><hr />")
text_1 = widgets.HTML('<br /><p style=\"color:#34568B;margin-left:30px;font-size: 15px;\">Find the Risk Factors from Hurricanes/Floods for Homes and Communities based on your Zip-code. <br /> Simple interface based on historical data from HURDAT2 and FEMA Databases! </p>')

text_desc = widgets.HTML(value="<br /><p style=\"margin-left:10%;font-size: 15px;\"> Start with entering your zipcode.. </p>")
text_break = widgets.HTML(value="<br />")

#file = open("pics/atl.png", "rb")
#im_atl = file.read()
#image_0 = widgets.Image(
#                    value=im_atl,
#                    format='png',
#                    width='100'
#                )

display(widgets.HTML('<style> \
    .header_box{ \
        height: 250px; \
        background-image: url(\'pics/atl1.png\'); \
        background-repeat: repeat-x; \
    } \
    .second_box{ \
        height: auto; \
        border-style: solid; \
        border-color: red; \
    } \
    .mystyle { \
        border-collapse: collapse; \
        margin: 25px 0; \
        font-size: 0.9em; \
        font-family: sans-serif; \
        min-width: 400px; \
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.15); \
        text-align: center; \
        padding: 5px; \
    } \
    </style> \
    '))


In [ ]:
### button event for showing the map - first button
btn_0 = widgets.Button(
                description='Run Analysis',
                tooltip='click here',
                style={'description_width': 'initial'}
                ,layout=widgets.Layout(height = '50px', margin='0% 0% 0% 10% ')
            )

btn_1 = widgets.Button(
                description='Clear Output',
                tooltip='click here',
                style={'description_width': 'initial'}
                ,layout=widgets.Layout(height = '50px', margin='0% 0% 0% 10% ')
            )


out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
def button_0_showmap(event):
    with out1:
        clear_output()
        ### use the input to update tables
        zip_input = zip_text.value
        lat_user = zip_df[zip_df['zip'] == zip_input]['latitude'].values[0]
        long_user = zip_df[zip_df['zip'] == zip_input]['longitude'].values[0]
        user_input = (lat_user, long_user)
        storm_df['Distance'] = storm_df.apply(lambda row: distance_calc(row, user_input), axis = 1)
        storm_df['Label'] = storm_df.apply(lambda row: label(row), axis = 1)
        ### put the plot output here
        map_USA = folium.Map(location=[lat_user, long_user], zoom_start = 8, scrollWheelZoom=False)
        hun_mile_df = storm_df[storm_df['Distance']<=75]
        hun_mile_riskr_df = hun_mile_df.sort_values('Distance')
        hun_mile_riskr_df = hun_mile_riskr_df[hun_mile_riskr_df['postcode'].notnull()].head(25)
        hun_mile_riskr_df = hun_mile_riskr_df[['postcode','Distance','county','state','POPULATION','AREA','BUILDVALUE','AGRIVALUE','HRCN_RISKR','CFLD_RISKR','RFLD_RISKR','FLOODRISK_LEVEL']]
        hun_mile_riskr_df.rename(columns={'postcode': 'zipcode', 'HRCN_RISKR': 'Hurricane_Risk_Rating', 'CFLD_RISKR': 'CoastalFlood_Risk_Rating', 'RFLD_RISKR': 'RiverFlood_Risk_Rating'}, inplace=True)
        locations = hun_mile_df[['Lat Split', 'Long Split']]
        locationlist = locations.values.tolist()
        userzip = folium.FeatureGroup(name='Your Zip Code')
        closestzip = folium.FeatureGroup(name='Neighboring Zip Codes w/ Storms')
        userzip.add_to(map_USA)
        closestzip.add_to(map_USA)
        folium.LayerControl().add_to(map_USA)
        for point in range(0, len(locationlist)):
            folium.Marker(locationlist[point], popup=storm_df['Label'][point]).add_to(closestzip)
        folium.Marker([lat_user, long_user], popup='Your Zipcode', icon=folium.Icon(color='red', icon = 'home')).add_to(userzip)
        #folium.Circle([lat_user, long_user], fill_color='#008080', radius=75).add_to(map_USA)
        display(widgets.HTML('<hr /><b><p style=\"color:darkgreen;font-size: 15px;\">Map showing User ZipCode along with historical storm locations around the area. <br /> Use the Map Control to toggle the markers!  </p></b>'))
        display(map_USA)
    with out2:
        clear_output()
        ### map the image to the code for max_wind_speed
        max_wind = hun_mile_df['Maximum Wind'].max()
        out2_desc = widgets.HTML('<br /><hr /><b><p style=\"color:brown;font-size: 15px;\">The Maximum wind-speed potential based on historical data is ')
        out2_desc.value = out2_desc.value + str(max_wind)
        if max_wind <= 25:
            image_max = "pics/House_0_25.png"
            out2_desc.value = out2_desc.value + " mph.</p><p style=\"color:darkgreen;font-size: 15px;\">Max: Wind Speeds 0-25 mph <br /> Winds this low are unlikely to cause any notable damage. </p></b>"
        elif max_wind <= 75:
            image_max = "pics/House_25_75.png"
            out2_desc.value = out2_desc.value + " mph.</p><p style=\"color:navy;font-size: 15px;\">Max: Wind Speeds 25-75 mph <br /> Shingles blown off. Tree limbs and other debris will be picked up. </p></b>"
        elif max_wind <= 100:
            image_max = "pics/House_75_100.png"
            out2_desc.value = out2_desc.value + " mph.</p><p style=\"color:orange;font-size: 15px;\">Max: Wind Speeds 75-100 mph <br /> More Damaging. Trees will be downed, mobile homes could be destroyed. Large projectiles will picked up and tossed. </p></b>"
        else:
            image_max = "pics/House_100_more.png"
            out2_desc.value = out2_desc.value + " mph.</p><p style=\"color:red;font-size: 15px;\">Max: Wind Speeds 100+ mph <br /> Extensive damage. Downed trees everywhere, major damage to roof and siding. Windows may be blown out. </p></b>"
        display(out2_desc)
        display(Image(filename=image_max, width=600))
    with out3:
        clear_output()
        ### map the image to the flood risk numbers
        pd.options.display.float_format = '{:.0f}'.format
        hun_mile_riskr_df_summ = hun_mile_riskr_df[['POPULATION','AREA','BUILDVALUE','AGRIVALUE','FLOODRISK_LEVEL']]
        hun_mile_riskr_df_summ.rename(columns={'POPULATION': 'Average Population', 'AREA': 'Average Land Area', 'BUILDVALUE': 'Average Building Value', 'AGRIVALUE': 'Average Agricultural Land Value', 'FLOODRISK_LEVEL': 'Average FloodRisk Rating'}, inplace=True)
        out3_table_desc = widgets.HTML('<br /><hr /><b><p style=\"color:brown;font-size: 15px;\">Average Parameter values around the ZipCode entered - Impact of damage and exposure depends on these variables in the model. </p></b>')
        out3_table = widgets.HTML(hun_mile_riskr_df_summ.mean().to_frame().to_html(classes='mystyle',header=False),layout=widgets.Layout(margin='0% 0% 0% 15% ')) 
        out3_table2_desc = widgets.HTML('<br /><b><p style=\"color:brown;font-size: 15px;\">Risk ratings for communities right closest to the ZipCode entered. This gives an idea of impact range and exposure area overall for community. </p></b>')
        hun_mile_riskr_df_summ2 = hun_mile_riskr_df[['zipcode','county','state','Hurricane_Risk_Rating','CoastalFlood_Risk_Rating','RiverFlood_Risk_Rating','FLOODRISK_LEVEL']] #.rename(columns={'zipcode': 'ZipCode', 'county': 'County','state': 'State'}, inplace=True)
        out3_table2 = widgets.HTML(hun_mile_riskr_df_summ2.head(5).to_html(classes='mystyle',index=False),layout=widgets.Layout(margin='0% 0% 0% 5% '))
        fldrisk_avg = hun_mile_riskr_df['FLOODRISK_LEVEL'].mean()
        out3_desc = widgets.HTML('<br /><hr /><b><p style=\"color:brown;font-size: 15px;\">The Average Flood-Risk Level expected based on historical data is </p>')
        if fldrisk_avg <= 1:
            image_avg = "pics/Surge_1-3.png"
            out3_desc.value = out3_desc.value + "<p style=\"color:darkgreen;font-size: 15px;\">Relatively Low to Very Low. <br /> Pay Attention to localized overflow!  </p></b>"
        elif fldrisk_avg <= 2:
            image_avg = "pics/Surge_4-6.png"
            out3_desc.value = out3_desc.value + "<p style=\"color:navy;font-size: 15px;\">Relatively Moderate. <br /> Expect flooding under the bridges and damage to roads!  </p></b>"
        elif fldrisk_avg <= 3:
            image_avg = "pics/Surge_7-9.png"
            out3_desc.value = out3_desc.value + "<p style=\"color:orange;font-size: 15px;\">Relatively High. <br /> Potential for flooding into homes and damaging overflow expected! Take Caution! </p></b>"
        else:
            image_avg = "pics/Surge_10-plus.png"
            out3_desc.value = out3_desc.value + "<p style=\"color:red;font-size: 15px;\">Very High! <br /> High Risk of Floods causing building damages and may completely submerge low-lying areas - Take Caution!  </p></b>"
        display(out3_desc)
        display(Image(filename=image_avg, width=1024))
        display(out3_table_desc)
        display(out3_table)
        display(out3_table2_desc)
        display(out3_table2)

def button_1_clear(event):
    with out1:
        clear_output()
    with out2:
        clear_output()
    with out3:
        clear_output()
        
btn_0.on_click(button_0_showmap)
btn_1.on_click(button_1_clear)

In [ ]:
### need to call the total widget box with components as needed
#vbox_0 = widgets.VBox(children=[image_0], layout=widgets.Layout(align_content='stretch'))
vbox_1 = widgets.VBox([text_0, text_1])
vbox_1.add_class("header_box")
hbox_1 = widgets.HBox([btn_0, btn_1])
vbox_2 = widgets.VBox([text_desc, zip_text, text_break, hbox_1, text_break])
vbox_3 = widgets.VBox([out1, out2, out3])
vbox_3.add_class("second_box")
page = widgets.VBox(children=[vbox_1, vbox_2, vbox_3],layout=widgets.Layout(width='80%',margin='2% 2% 2% 2% ',align_self='center'))
display(page)

In [1]:
### end
#!pip freeze > requirements.txt